This notebook deals with the microfading rawdata obtained when performing analyses with the microfading device from Fotonowy. It first describes the rawdata obtained and then provided python code to process them in order to obtain interim MFT data.

# Import statements 

In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
import seaborn as sns
import colour
import scipy.interpolate as sip 

from scipy.optimize import curve_fit 

import process_rawdata

/home/gus/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
cd /home/gus/Documents/RCE/projects/2023-8005_FL_REL_MFT-Fotonowy/data/raw/MFT/

/home/gus/Documents/RCE/projects/2023-8005_FL_REL_MFT-Fotonowy/data/raw/MFT


# Rawdata description 

In [3]:
rawfiles = sorted(glob('*G03_07*'))
rawfiles

['noProject-BWS0001-BW1-G03_07-spect.txt',
 'noProject-BWS0001-BW1-G03_07-spect_convert.txt',
 'noProject-BWS0001-BW1-G03_07.rfc',
 'noProject-BWS0001-BW1-G03_07.txt']

Each microfading analyses performed with the Fotonowy device returns 4 files:
* a **.txt** file: contains the colorimetric info (dE equations + Lab) as a function of time and energy, as well as some metadata
* a **.rfc** file: contains the inital and final reflectance spectra
* a **-spect.txt** file: contains the raw signals (counts) recorded by the spectrometer (white, dark references and measurements on object) as a function of wavelengths
* a **-spect_convert.txt** file: same as the *-spect.txt* file but with a reduced wavelength range (380-780 nm)

The reflectance spectra between the first and last measurements are not provided. In order to retrieve them, one will need to calculate them using the raw signals provided in the *-spect.txt* and *-spect_convert.txt* files. 

In [4]:
txt_file = 'noProject-BWS0001-BW1-G03_07.txt'
txt_df = pd.read_csv(txt_file, sep='\t', skiprows = 8)

rfc_file = 'noProject-BWS0001-BW1-G03_07.rfc'
rfc_df = pd.read_csv(rfc_file, sep='\t', skiprows=2)

spect_file = 'noProject-BWS0001-BW1-G03_07-spect.txt'
spect_df = pd.read_csv(spect_file, sep='\t', skiprows=1)

spect_convert_file = 'noProject-BWS0001-BW1-G03_07-spect_convert.txt'
spect_convert_df = pd.read_csv(spect_convert_file, sep='\t', skiprows=1)

In [7]:
rawfiles = sorted(glob('*G03_07*'))
files = sorted(glob('*-spect_convert.txt*'))
a = process_rawdata.MFT_Fotonowy(rawfiles,info_analysis=True, save=False, return_data=True, )

In [8]:
a[1]

,1,2,3,4,5,6,7,8,9
parameter,,,,,,,,,
Date,Thu Nov 9 15:14:09 2023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Object,noProject-BWS0001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Illuminant,D65,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Sample integration time [ms],50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
White standard integration time [ms],50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
587.07,11.4,922.83,52.91,-0.98,-35.79,35.8,268.44,8.97,2.55
592.22,11.5,930.92,52.95,-0.9,-35.83,35.84,268.56,8.92,2.58
597.36,11.6,939.02,53.0,-0.8,-35.9,35.91,268.73,8.82,2.61


In [9]:
a[0]

He_MJ/m2,0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,...,10.9,11.0,11.1,11.2,11.3,11.4,11.5,11.6,11.7,11.8
wavelength_nm,,,,,,,,,,,,,,,,,,,,,
410,0.3897,0.3722,0.3711,0.4123,0.4211,0.3670,0.3397,0.3528,0.3638,0.3723,...,0.3711,0.3795,0.3832,0.3819,0.3893,0.4042,0.4142,0.4205,0.4080,0.3850
411,0.4005,0.4261,0.4461,0.4516,0.4415,0.4010,0.3690,0.3500,0.3573,0.3972,...,0.3808,0.3908,0.3927,0.3855,0.3919,0.4098,0.4223,0.4311,0.4157,0.3870
412,0.3971,0.3970,0.3957,0.3916,0.3872,0.3822,0.3884,0.4119,0.4222,0.4163,...,0.4038,0.3902,0.3822,0.3802,0.3759,0.3697,0.3746,0.3873,0.3891,0.3848
413,0.4220,0.4304,0.4343,0.4265,0.4161,0.4004,0.3959,0.4083,0.4153,0.4154,...,0.3811,0.4014,0.3969,0.3656,0.3637,0.3874,0.4089,0.4291,0.4223,0.4007
414,0.4369,0.4377,0.4356,0.4260,0.4183,0.4142,0.4130,0.4162,0.4170,0.4147,...,0.4175,0.4277,0.4241,0.4055,0.3896,0.3759,0.3858,0.4122,0.4197,0.4168
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
746,1.1833,1.1929,1.1930,1.1687,1.1555,1.1641,1.1749,1.1891,1.1757,1.1278,...,1.0693,1.0841,1.0787,1.0512,1.0531,1.0802,1.0894,1.0860,1.0811,1.0753
747,1.2408,1.2595,1.2613,1.2202,1.1899,1.1807,1.1809,1.1955,1.2204,1.2582,...,1.1231,1.1064,1.0981,1.0991,1.1040,1.1126,1.1405,1.1822,1.1635,1.1115
748,1.2387,1.2573,1.2662,1.2501,1.2233,1.1753,1.1543,1.1739,1.1902,1.2027,...,1.1074,1.0728,1.0647,1.0853,1.1106,1.1397,1.1629,1.1818,1.1643,1.1267


In [202]:
a[1]

,0,1
0,Date,Thu Nov 9 15:14:09 2023
1,Object,noProject-BWS0001
2,Illuminant,D65
3,Sample integration time [ms],50
4,White standard integration time [ms],50
5,Sample,07
6,Curr,"700 (Watts: 0.0194186, MLuxes: 5.65894)"
7,LED,1
8,[MEASUREMENT DATA],


In [203]:
pd.concat([a[1],a[0]])[:20]

,0,1,2,3,4,5,6,7,8,9
0,Date,Thu Nov 9 15:14:09 2023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Object,noProject-BWS0001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Illuminant,D65,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Sample integration time [ms],50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,White standard integration time [ms],50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Sample,07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Curr,"700 (Watts: 0.0194186, MLuxes: 5.65894)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,LED,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,[MEASUREMENT DATA],,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,time_s,He_MJ/m2,Hv_klxh,L*,a*,b*,C*,h,dE76,dE00


In [172]:
b

,0,1,2,3,4,5,6,7,8,9
index,time_s,He_MJ/m2,Hv_klxh,L*,a*,b*,C*,h,dE76,dE00
0,0.0,0.0,0.0,50.91,2.36,-43.87,43.93,273.08,0.0,0.0
1,5.15,0.1,8.09,51.05,2.05,-43.46,43.51,272.7,0.53,0.17
2,10.3,0.2,16.19,51.15,1.83,-43.1,43.14,272.43,0.96,0.3
3,15.45,0.3,24.28,51.17,1.8,-42.92,42.95,272.4,1.14,0.33
...,...,...,...,...,...,...,...,...,...,...
114,587.07,11.4,922.83,52.91,-0.98,-35.79,35.8,268.44,8.97,2.55
115,592.22,11.5,930.92,52.95,-0.9,-35.83,35.84,268.56,8.92,2.58
116,597.36,11.6,939.02,53.0,-0.8,-35.9,35.91,268.73,8.82,2.61
117,602.51,11.7,947.11,52.93,-0.76,-36.04,36.05,268.79,8.66,2.53


In [177]:
len(a)

119

In [111]:
d = a.T.reset_index().T

In [114]:
d.index = np.arange(0,120)

In [115]:
d

,0,1,2,3,4,5,6,7,8,9
0,time_s,He_MJ/m2,Hv_klxh,L*,a*,b*,C*,h,dE76,dE00
1,0.0,0.0,0.0,50.91,2.36,-43.87,43.93,273.08,0.0,0.0
2,5.15,0.1,8.09,51.05,2.05,-43.46,43.51,272.7,0.53,0.17
3,10.3,0.2,16.19,51.15,1.83,-43.1,43.14,272.43,0.96,0.3
4,15.45,0.3,24.28,51.17,1.8,-42.92,42.95,272.4,1.14,0.33
...,...,...,...,...,...,...,...,...,...,...
115,587.07,11.4,922.83,52.91,-0.98,-35.79,35.8,268.44,8.97,2.55
116,592.22,11.5,930.92,52.95,-0.9,-35.83,35.84,268.56,8.92,2.58
117,597.36,11.6,939.02,53.0,-0.8,-35.9,35.91,268.73,8.82,2.61
118,602.51,11.7,947.11,52.93,-0.76,-36.04,36.05,268.79,8.66,2.53


In [44]:
raw_file_dE

'noProject-BWS0001-BW1-G03_09.txt'

In [35]:
for file in files:
    raw_file_dE = file.replace('-spect_convert.txt', '.txt') 

    raw_file_path = Path(raw_file_dE)
    print(raw_file_path.name)

noProject-BWS0001-BW1-G03_03.txt
noProject-BWS0001-BW1-G03_04.txt
noProject-BWS0001-BW1-G03_05.txt
noProject-BWS0001-BW1-G03_06.txt
noProject-BWS0001-BW1-G03_07.txt
noProject-BWS0001-BW1-G03_08.txt
noProject-BWS0001-BW1-G03_09.txt


In [155]:
lookfor = '#Time'
f = open(raw_file_dE).read()

parameters = f[:f.index(lookfor)].splitlines()

dic_parameters = {}

for i in parameters:   
    
    key = i[2:i.index(':')]
    value = i[i.index(':')+2:]

    dic_parameters[key]=[value]
    df_parameters = pd.DataFrame.from_dict(dic_parameters).T
    
#df_parameters.columns = ['He_MJ/m2']            
#df_parameters.index.names = ['time_s'] 

#df_parameters = df_parameters.rename(index={'Date':'date_time'})                 
#df_parameters = df_parameters.reset_index()

In [146]:
df_parameters.loc[:,['[MEASUREMENT DATA]']] = ''

In [157]:
df_parameters.loc['[MEASUREMENT DATA]'] = ''

In [153]:
df_parameters.loc['8'] = ['hh','']

In [158]:
df_parameters

,0
Date,Thu Nov 9 15:43:33 2023
Object,noProject-BWS0001
Illuminant,D65
Sample integration time [ms],50
White standard integration time [ms],50
Sample,09
Curr,"700 (Watts: 0.0194186, MLuxes: 5.65894)"
LED,1
[MEASUREMENT DATA],


In [136]:
c = pd.concat([df_parameters, d], axis=0)

In [138]:
c[:20]

,index,0,1,2,3,4,5,6,7,8
0,Date,Thu Nov 9 15:43:33 2023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Object,noProject-BWS0001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Illuminant,D65,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Sample integration time [ms],50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,White standard integration time [ms],50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Sample,09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Curr,"700 (Watts: 0.0194186, MLuxes: 5.65894)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,LED,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,time_s,He_MJ/m2,Hv_klxh,L*,a*,b*,C*,h,dE76,dE00
1,0.0,0.0,0.0,50.91,2.36,-43.87,43.93,273.08,0.0,0.0


In [133]:
d.columns = ['index', 0,1,2,3,4,5,6,7,8]

In [134]:
d

,index,0,1,2,3,4,5,6,7,8
0,time_s,He_MJ/m2,Hv_klxh,L*,a*,b*,C*,h,dE76,dE00
1,0.0,0.0,0.0,50.91,2.36,-43.87,43.93,273.08,0.0,0.0
2,5.15,0.1,8.09,51.05,2.05,-43.46,43.51,272.7,0.53,0.17
3,10.3,0.2,16.19,51.15,1.83,-43.1,43.14,272.43,0.96,0.3
4,15.45,0.3,24.28,51.17,1.8,-42.92,42.95,272.4,1.14,0.33
...,...,...,...,...,...,...,...,...,...,...
115,587.07,11.4,922.83,52.91,-0.98,-35.79,35.8,268.44,8.97,2.55
116,592.22,11.5,930.92,52.95,-0.9,-35.83,35.84,268.56,8.92,2.58
117,597.36,11.6,939.02,53.0,-0.8,-35.9,35.91,268.73,8.82,2.61
118,602.51,11.7,947.11,52.93,-0.76,-36.04,36.05,268.79,8.66,2.53


In [135]:
df_parameters

,index,0
0,Date,Thu Nov 9 15:43:33 2023
1,Object,noProject-BWS0001
2,Illuminant,D65
3,Sample integration time [ms],50
4,White standard integration time [ms],50
5,Sample,09
6,Curr,"700 (Watts: 0.0194186, MLuxes: 5.65894)"
7,LED,1


In [120]:
d

,0,1,2,3,4,5,6,7,8,9
0,time_s,He_MJ/m2,Hv_klxh,L*,a*,b*,C*,h,dE76,dE00
1,0.0,0.0,0.0,50.91,2.36,-43.87,43.93,273.08,0.0,0.0
2,5.15,0.1,8.09,51.05,2.05,-43.46,43.51,272.7,0.53,0.17
3,10.3,0.2,16.19,51.15,1.83,-43.1,43.14,272.43,0.96,0.3
4,15.45,0.3,24.28,51.17,1.8,-42.92,42.95,272.4,1.14,0.33
...,...,...,...,...,...,...,...,...,...,...
115,587.07,11.4,922.83,52.91,-0.98,-35.79,35.8,268.44,8.97,2.55
116,592.22,11.5,930.92,52.95,-0.9,-35.83,35.84,268.56,8.92,2.58
117,597.36,11.6,939.02,53.0,-0.8,-35.9,35.91,268.73,8.82,2.61
118,602.51,11.7,947.11,52.93,-0.76,-36.04,36.05,268.79,8.66,2.53


In [102]:
a

,time_s,He_MJ/m2,Hv_klxh,L*,a*,b*,C*,h,dE76,dE00
0,0.00,0.0,0.00,50.91,2.36,-43.87,43.93,273.08,0.00,0.00
1,5.15,0.1,8.09,51.05,2.05,-43.46,43.51,272.70,0.53,0.17
2,10.30,0.2,16.19,51.15,1.83,-43.10,43.14,272.43,0.96,0.30
3,15.45,0.3,24.28,51.17,1.80,-42.92,42.95,272.40,1.14,0.33
4,20.60,0.4,32.38,51.20,1.69,-42.67,42.70,272.27,1.41,0.37
...,...,...,...,...,...,...,...,...,...,...
114,587.07,11.4,922.83,52.91,-0.98,-35.79,35.80,268.44,8.97,2.55
115,592.22,11.5,930.92,52.95,-0.90,-35.83,35.84,268.56,8.92,2.58
116,597.36,11.6,939.02,53.00,-0.80,-35.90,35.91,268.73,8.82,2.61
117,602.51,11.7,947.11,52.93,-0.76,-36.04,36.05,268.79,8.66,2.53


In [81]:
a.loc[len(a)] = a.columns
a.columns = range(len(a.columns))

In [82]:
a

,0,1,2,3,4,5,6,7,8
time_s,,,,,,,,,
0.00,0.0,0.0,50.91,2.36,-43.87,43.93,273.08,0.0,0.0
5.15,0.1,8.09,51.05,2.05,-43.46,43.51,272.7,0.53,0.17
10.30,0.2,16.19,51.15,1.83,-43.1,43.14,272.43,0.96,0.3
15.45,0.3,24.28,51.17,1.8,-42.92,42.95,272.4,1.14,0.33
20.60,0.4,32.38,51.2,1.69,-42.67,42.7,272.27,1.41,0.37
...,...,...,...,...,...,...,...,...,...
592.22,11.5,930.92,52.95,-0.9,-35.83,35.84,268.56,8.92,2.58
597.36,11.6,939.02,53.0,-0.8,-35.9,35.91,268.73,8.82,2.61
602.51,11.7,947.11,52.93,-0.76,-36.04,36.05,268.79,8.66,2.53


In [50]:
parameters

['# Date: Thu Nov 9 15:43:33 2023',
 '# Object: noProject-BWS0001',
 '# Illuminant: D65',
 '# Sample integration time [ms]: 50',
 '# White standard integration time [ms]: 50',
 '# Sample: 09',
 '# Curr: 700 (Watts: 0.0194186, MLuxes: 5.65894)',
 '# LED: 1']

In [30]:
from pathlib import Path

# Process rawdata 

 In the following cell, the microfading raw are being processed in order to obtain what we called 'interim data', based on standard file structure. The interim data are composed of two Pandas dataframes that can be saved as csv files: 
 
 1. **df_sp** : contains the reflectance spectra
 2. **df_dE** : contains the colorimetric data (LabCh, dE equations) according to time and energy values

In [ ]:
# Import the rawdata files into two Pandas dataframes
raw_file_sp = 'noProject-BWS0001-BW1-G03_07-spect_convert.txt'
raw_df_sp = pd.read_csv(raw_file_sp, sep='\t', skiprows = 1)

raw_file_dE = 'noProject-BWS0001-BW1-G03_07.txt'
raw_df_dE = pd.read_csv(raw_file_dE, sep='\t', skiprows = 8)

# final wavelength range
wanted_wl = pd.Index(np.arange(410,751), name='wavelength_nm') 

# interpolated spectral counts values
df_counts_interpolated = pd.DataFrame(index=wanted_wl)

for col in raw_df_sp.columns:
    counts_interpolated = sip.interp1d(x=raw_df_sp.index, y=raw_df_sp[col])(wanted_wl)
    df_counts_interpolated[col] = counts_interpolated

# select white and dark spectral references (first and second columns respectively)
white_ref = df_counts_interpolated.iloc[:,0].values
dark_ref = df_counts_interpolated.iloc[:,1].values

# remove the white and dark ref
cols = df_counts_interpolated.columns
df_counts = df_counts_interpolated.iloc[:,2:-1]
df_counts.columns = cols[3:]
df_counts.index.name = 'wavelength_nm'

# define parameters for colorimetric calculations
D65 = colour.SDS_ILLUMINANTS["D65"]
d65 = colour.CCS_ILLUMINANTS["cie_2_1931"]["D65"]
cmfs = colour.colorimetry.MSDS_CMFS_STANDARD_OBSERVER["CIE 1931 2 Degree Standard Observer"]

# calculate the initial Lab values
counts_i = df_counts.iloc[:,0]
sp_i = counts_i.values/(white_ref)
sd_i = colour.SpectralDistribution(sp_i, wanted_wl)
XYZ_i = colour.sd_to_XYZ(sd_i, cmfs, illuminant=D65)
Lab_i = np.round(colour.XYZ_to_Lab(XYZ_i / 100, d65),2)

# final reflectance values
df_sp = pd.DataFrame(index=wanted_wl)
df_sp.index.name = 'wavelength_nm'

# empty list to store XYZ values
XYZ = []

# drop the before last column of df_counts
df_counts = df_counts.drop(df_counts.iloc[:,-2].name,axis=1)

# compute the reflectance, XYZ, and LabCh values
for col in df_counts.columns:  
    counts = df_counts[col].values
    sp = counts / white_ref
    sd = colour.SpectralDistribution(sp, wanted_wl)
    df_sp[col[15:]] = sp
    XYZ.append(colour.sd_to_XYZ(sd, cmfs, illuminant=D65))

XYZ = np.array(XYZ)

Lab = np.array([colour.XYZ_to_Lab(d / 100, d65) for d in XYZ])
LCh = np.array([colour.Lab_to_LCHab(d) for d in Lab])
    
L = []
a = []
b = []
C = []
h = []

[L.append(np.round(i[0],3)) for i in Lab]
[a.append(np.round(i[1],3)) for i in Lab]
[b.append(np.round(i[2],3)) for i in Lab]
[C.append(np.round(i[1],3)) for i in LCh]
[h.append(np.round(i[2],3)) for i in LCh]


# Compute the delta E values
dE76 = np.round(np.array([colour.delta_E(Lab[0], d, method="CIE 1976") for d in Lab]),3)
dE00 = np.round(np.array([colour.delta_E(Lab[0], d) for d in Lab]),3)

# Retrieve the times and energy values
times = [float(col[15:-3]) for col in df_counts.columns]
He = raw_df_dE['Watts']       # in MJ/m²
Hv = raw_df_dE['Lux'] * 1000  # in klxh


# interpolate colorimetric data
wanted_He = np.arange(0,He.values[-1], 0.1)

times = np.round(sip.interp1d(x=He, y= times)(wanted_He),2)
Hv = np.round(sip.interp1d(x=He, y= Hv)(wanted_He),2)
L = np.round(sip.interp1d(x=He, y= L)(wanted_He),2)
a = np.round(sip.interp1d(x=He, y= a)(wanted_He),2)
b = np.round(sip.interp1d(x=He, y= b)(wanted_He),2)
C = np.round(sip.interp1d(x=He, y= C)(wanted_He),2)
h = np.round(sip.interp1d(x=He, y= h)(wanted_He),2)
dE76 = np.round(sip.interp1d(x=He, y= dE76)(wanted_He),2)
dE00 = np.round(sip.interp1d(x=He, y= dE00)(wanted_He),2)


# Final colorimetric values
df_dE = pd.DataFrame({'time_s':times,'He_MJ/m2':wanted_He,'Hv_klxh':Hv, 'L*':L, 'a*':a, 'b*':b, 'C*':C, 'h':h, 'dE76':dE76, 'dE00':dE00})



df_sp.columns = He
df_sp = pd.DataFrame(data = np.round(sip.interp2d(df_sp.columns,pd.to_numeric(df_sp.index), df_sp)(wanted_He, wanted_wl),4),
                    index = wanted_wl,
                    columns = wanted_He,
                    )
df_sp.columns.name = 'He_MJ/m2'

In [ ]:
df_dE

In [ ]:
df_sp

In [ ]:
raw_df_dE

# Visualizations 

In [ ]:
# Calculated Fotonowy data from the .rfc file

data_fy = rfc_df[(rfc_df['#Wavelength'] > 409) & (rfc_df['#Wavelength'] < 751)]

wl_fy = data_fy['#Wavelength']
sp_i_fy = data_fy['#Inital reflectance [%]']
sp_f_fy = data_fy['#Final reflectance [%]']

# Given Lab data from Fotonowy .txt file
Fy_He = raw_df_dE['Watts']
Fy_L = raw_df_dE['L']
Fy_a = raw_df_dE['a']
Fy_b = raw_df_dE['b']

# Plots
sns.set()
fs = 24
fig1, ax1 = plt.subplots(1,1, figsize=(15,7))
fig2, [ax2,ax3,ax4] = plt.subplots(3,1, figsize=(15,15))

ax1.plot(wl_fy, sp_i_fy, ls='--', lw=3, color='b', label='Initial - Fotonowy')
ax1.plot(wl_fy, sp_f_fy, ls='--', lw=3, color='r', label='Final - Fotonowy')

ax1.plot(wl_rce, sp_i_rce, color='k', label='Initial - RCE')
ax1.plot(wl_rce, sp_f_rce, color='k', label='Final - RCE')

ax2.plot(Fy_He,Fy_L, ls='--', lw=3, color='b', label='Fotonowy')
ax3.plot(Fy_He,Fy_a, ls='--', lw=3, color='b', label='Fotonowy')
ax4.plot(Fy_He,Fy_b, ls='--', lw=3, color='b', label='Fotonowy')

ax2.plot(df_dE['He_MJ/m2'],df_dE['L*'], ls='-', lw=3, color='r', label='RCE')
ax3.plot(df_dE['He_MJ/m2'],df_dE['a*'], ls='-', lw=3, color='r', label='RCE')
ax4.plot(df_dE['He_MJ/m2'],df_dE['b*'], ls='-', lw=3, color='r', label='RCE')

ax1.set_xlabel('Wavelength $\lambda$ (nm)', fontsize=fs)
ax4.set_xlabel('Radiant exposure $H_e$ (MJ/m²)', fontsize=fs)

ax1.set_ylabel('Reflectance (%)', fontsize=fs)
ax2.set_ylabel('CIE $L^*$', fontsize=fs)
ax3.set_ylabel('CIE $a^*$', fontsize=fs)
ax4.set_ylabel('CIE $b^*$', fontsize=fs)

ax1.xaxis.set_tick_params(labelsize=fs)
ax2.xaxis.set_tick_params(labelsize=fs)
ax3.xaxis.set_tick_params(labelsize=fs)
ax4.xaxis.set_tick_params(labelsize=fs)

ax1.yaxis.set_tick_params(labelsize=fs)
ax2.yaxis.set_tick_params(labelsize=fs)
ax3.yaxis.set_tick_params(labelsize=fs)
ax4.yaxis.set_tick_params(labelsize=fs)


ax1.legend(fontsize=fs)
ax2.legend(fontsize=fs)
ax3.legend(fontsize=fs)
ax4.legend(fontsize=fs)

plt.tight_layout()
plt.show()

In [ ]:
# Reflectance spectra

D65 = colour.SDS_ILLUMINANTS["D65"]
d65 = colour.CCS_ILLUMINANTS["cie_2_1931"]["D65"]
cmfs = colour.colorimetry.MSDS_CMFS_STANDARD_OBSERVER["CIE 1931 2 Degree Standard Observer"]
wanted_wl = np.arange(410,751)

# Given Fotonowy data from txt file

fy_txt_file = 'noProject-BWS0001-BW1-G03_07.txt'
fy_txt_df = pd.read_csv(fy_txt_file, sep='\t', skiprows = 8)
Lab_i_fy_txt = fy_txt_df.iloc[0,:][['L','a','b']].values
Lab_f_fy_txt = fy_txt_df.iloc[-1,:][['L','a','b']].values

# Calculated Fotonowy data

data_fy = rfc_df[(rfc_df['#Wavelength'] > 409) & (rfc_df['#Wavelength'] < 751)]

wl_fy = data_fy['#Wavelength']
sp_i_fy = data_fy['#Inital reflectance [%]']
sp_f_fy = data_fy['#Final reflectance [%]']


sd_i_fy = colour.SpectralDistribution(sip.interp1d(x=wl_fy, y=sp_i_fy/100)(wanted_wl),wanted_wl)
XYZ_i_fy = colour.sd_to_XYZ(sd_i_fy, cmfs, illuminant=D65)
Lab_i_fy = np.round(colour.XYZ_to_Lab(XYZ_i_fy / 100, d65),2)

sd_f_fy = colour.SpectralDistribution(sip.interp1d(x=wl_fy, y=sp_f_fy/100)(wanted_wl),wanted_wl)
XYZ_f_fy = colour.sd_to_XYZ(sd_f_fy, cmfs, illuminant=D65)
Lab_f_fy = np.round(colour.XYZ_to_Lab(XYZ_f_fy / 100, d65),2)

# RCE data

wl_rce = df_sp.index
sp_i_rce = df_sp.iloc[:,0].values * 100
sp_f_rce = df_sp.iloc[:,-1].values * 100


# Lab dataframe

cols = [('Initial SP','Fy txt'), ('Initial SP', 'Fy calc'), ('Initial SP', 'RCE'), ('Final SP','Fy txt'), ('Final SP', 'Fy calc'), ('Final SP', 'RCE')]

df_Lab = pd.DataFrame(columns=cols, index = ['L*','a*','b*'])
df_Lab.columns = pd.MultiIndex.from_tuples(df_Lab.columns)

df_Lab['Initial SP','Fy txt'] = Lab_i_fy_txt
df_Lab['Initial SP','Fy calc'] = Lab_i_fy
df_Lab['Initial SP','RCE'] = Lab_i
df_Lab['Final SP','Fy txt'] = Lab_f_fy_txt
df_Lab['Final SP','Fy calc'] = Lab_f_fy
df_Lab['Final SP','RCE'] = np.round(Lab[-1],2)

# Plots
sns.set()
fs = 24
fig, ax = plt.subplots(1,1, figsize=(15,7))

ax.plot(wl_fy, sp_i_fy, ls='--', lw=3, color='b', label='Initial - Fotonowy')
ax.plot(wl_fy, sp_f_fy, ls='--', lw=3, color='r', label='Final - Fotonowy')

ax.plot(wl_rce, sp_i_rce, color='k', label='Initial - RCE')
ax.plot(wl_rce, sp_f_rce, color='k', label='Final - RCE')


plt.legend(fontsize=fs)
plt.tight_layout()
plt.show()

df_Lab


In [ ]:

# Import the rawfile
rawfile = 'noProject-BWS0001-BW1-G03_07-spect_convert.txt'
df = pd.read_csv(rawfile, sep='\t', skiprows = 1)

wl = df.index

dark_ref = df['White Std. spectrum'].values
white_ref = df['#Wavelength [nm]'].values

sp1 = df['Sample spectrum8.849[s]']
sp2 = df['Sample spectrum609.196[s]']



sp_list = [sp1,sp2]
t_list = []
L_list = []
a_list = []
b_list = []
dE_list = []
wanted_wl = np.arange(400,750)
i = 2

sns.set()
fs = 24
fig, (ax1,ax2) = plt.subplots(3,1,figsize=(15,15))

sp_i = df['Dark specetrum']
rs_i = sp_i.values/(white_ref)
rs_i = sip.interp1d(x=wl,y=rs_i)(wanted_wl)
sd_i = colour.SpectralDistribution(rs_i, wanted_wl)
D65 = colour.SDS_ILLUMINANTS["D65"]
d65 = colour.CCS_ILLUMINANTS["cie_2_1931"]["D65"]
cmfs = colour.colorimetry.MSDS_CMFS_STANDARD_OBSERVER["CIE 1931 2 Degree Standard Observer"]

XYZ_i = colour.sd_to_XYZ(sd_i, cmfs, illuminant=D65)
Lab_i = np.round(colour.XYZ_to_Lab(XYZ_i / 100, d65),2)

for col in df.columns[2:-1]:
    sp = df[col]
    
    rs = sp.values/(white_ref)
    rs_intp = sip.interp1d(x=wl,y=rs)(wanted_wl)
    
    sd = colour.SpectralDistribution(rs_intp, wanted_wl)
    D65 = colour.SDS_ILLUMINANTS["D65"]
    d65 = colour.CCS_ILLUMINANTS["cie_2_1931"]["D65"]
    cmfs = colour.colorimetry.MSDS_CMFS_STANDARD_OBSERVER["CIE 1931 2 Degree Standard Observer"]

    XYZ = colour.sd_to_XYZ(sd, cmfs, illuminant=D65)
    Lab = np.round(colour.XYZ_to_Lab(XYZ / 100, d65),2)
    
    dE = colour.delta_E(Lab_i, Lab)
    
    ax2.plot(wanted_wl,rs_intp)
    
    L_list.append(Lab[0])
    a_list.append(Lab[1])
    b_list.append(Lab[2])
    dE_list.append(dE)
    
    
    
    time = float(df.columns[i+1][15:-3])
    t_list.append(time)
    i = i+1
    
ax1.plot(wl, white_ref, ls='--', label='White ref')
ax1.plot(wl, dark_ref, ls='--', label='Dark ref')
ax1.plot(wl,sp1, lw=3, label='initial sp')
ax1.plot(wl,sp2, lw=3, label='final sp')

ax1.set_xlim(400,750)
ax2.set_xlim(400,750)
ax2.set_ylim(0,1)

ax1.xaxis.set_tick_params(labelsize=fs)
ax2.xaxis.set_tick_params(labelsize=fs)

ax1.yaxis.set_tick_params(labelsize=fs)
ax2.yaxis.set_tick_params(labelsize=fs)

ax1.set_xlabel('Wavelength $\lambda$ (nm)', fontsize=fs)
ax2.set_xlabel('Wavelength $\lambda$ (nm)', fontsize=fs)

ax1.set_ylabel('Counts', fontsize=fs)
ax2.set_ylabel('Reflectance', fontsize=fs)


ax1.legend(fontsize=fs)
plt.show()

# Corrections

* In the **.txt** file, provide the $C^*$ and $h$ values so that people don't have to calculate them
* In the **.txt** file, change the columns name, *watts* -> *MJ/m²* and *lux* -> *Mlxh*, could maybe also add official symbols ($H_e$ and $H_v$)
* In the **-spect_convert.txt**, spelling mistake: *Dark specetrum* -> *Dark spectrum*

In [ ]:
df_dE

In [ ]:
x = wanted_He
y = dE00

c0 = 0.1
c1 = 1
c2 = 0.0001

# define models called f    
def fitting(x,c0,c1,c2):
    f = c0*(1/(1+(c1*np.exp(-c2*x)))) - (c0/c1)+1
    return f


def fitting2(x,c0,c1):
    f = c0*(1-np.exp(-c1*x))
    return f
    

    # find the best parameters for the model f
x_ext = np.arange(0,20,1)

c,cov = curve_fit(fitting,x,y,p0 = [c0,c1,c2])
y_ext = fitting(x_ext,c[0],c[1],c[2])

#c2,cov2 = curve_fit(fitting2,x,y,p0 = [c0,c1])
#y_ext2 = fitting2(x_ext,c[0],c[1])

plt.plot(wanted_He, y, label='measured')
plt.plot(x_ext,y_ext, label='fitted')
#plt.plot(x_ext,y_ext2, label='fitted2')

plt.legend()

In [ ]:
c

In [ ]:
c2

In [ ]:
y_ext

In [ ]:
y_ext2

In [ ]:
file = 'noProject-BWS0001-BW1-G03_07-spect.txt'
df = pd.read_csv(file, sep='\t', skiprows=1)
wl = df.T.index.astype(float)

fig, ax = plt.subplots(1,1, figsize=(15,10))

i = 1
for c in df.T.columns:
    
    sp = df.T[c]
    
    ax.plot(wl, sp)
    
    i = i +1


#ax.set_xlim(300,800)
ax.set_ylim(0,4000)
    
plt.show()
print(i)

In [ ]:
wl_fy

In [ ]:
project = '2023-8005'
process_raw.MFT_Fotonowy(project=project)

In [ ]:
file = 'noProject-BWS0001-BW1-G03_05-spect.txt'



In [ ]:
df

In [ ]:
df

In [ ]:
file = 'noProject-BWS0001-BW1-G03_05-spect_convert.txt'

df = pd.read_csv(file, sep='\t', skiprows=1)

wl = df.index.astype(float)
sp_i = df['Dark specetrum']
sp_f = df['Sample spectrum0[s]']

df

fig, ax = plt.subplots(1,1, figsize=(15,10))

ax.plot(wl, sp_i)
ax.plot(wl, sp_f)

In [ ]:
file = 'noProject-BWS0001-BW1-G03_05.rfc'

df = pd.read_csv(file, sep='\t', skiprows=2)

wl = df['#Wavelength']
sp_i = df['#Inital reflectance [%]']
sp_f = df['#Final reflectance [%]']



fig, ax = plt.subplots(1,1, figsize=(15,10))

ax.plot(wl, sp_i)
ax.plot(wl, sp_f)

ax.set_xlim(400, 750)

In [ ]:
df['#Inital reflectance [%]']

In [ ]:
df

In [ ]:
file = 'noProject-BWS0001-BW1-G03_05.txt'

df = pd.read_csv(file, sep='\t', skiprows=8)
H_foto = df['Watts']
dE_foto = df['dE2000']

itm = MFT.MFT(Id='MF.BWS0004.01', project='2023-9001')
itm_Lab = itm.get_data(data='Lab')

H_itm = itm_Lab.index
dE_itm = itm_Lab['dE00']


LB = Lightbox.LB('0056')
LB_data = LB.get_data(background='wbg')[1]

H_LB = LB_data.index.get_level_values('cum_MJ/m2')
dE_LB = LB_data['mean']['dE00']

sns.set()
fig, ax = plt.subplots(1,1, figsize=(15,9))

ax.plot(H_foto,dE_foto)
ax.plot(H_itm,dE_itm, ls='--')
ax.plot(H_LB,dE_LB, ls='-.')

ax.set_xlim(0,20)
ax.set_ylim(0,10)

plt.show()

In [ ]:
d = 0.59 # mm
p = 5.2  # mW

A = 3.14 * ((d/1000/2)**2)
H = (p/1000) / A

H